# WISE2MBH Tutorial

In [6]:
import wise2mbh as wm
from astropy.table import Table

In [8]:
allwise = Table.read('../samples/AllWISE_sample.fits')

In [10]:
wm.param_montecarlo(10,0.1)

NameError: name 'number' is not defined